# Instrucciones

In [1]:
from apifunctions import *



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/danigomezlechonbarrachina/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [2]:
sales_growth=0.08
gross_margin="same as last year"
Personnel_expenses="same as last year"
interest_on_cash=0.005
tax_rate="same as last year"
DSO="same as last year"
DIO="same as last year"
DPO="same as last year"
depreciation_tangible=0.06
amortization_intangible=0.1
capex_tangible=0.05
capex_intangible=0.02
dividends=0
interest_rate_on_debt="same as last year"
#maturities

In [3]:
years=["2018","2019","2020","2021"]

Cargamos los financial statements:

In [4]:
%store -r income_statement
%store -r current_assets
%store -r non_current_assets
%store -r current_liabilities
%store -r non_current_liabilities
%store -r equity
%store -r debt_details

In [5]:
# Adding the columns for the following next years

new_years=["2022E","2023E","2024E","2025E","2026E"]
dataframes=[income_statement,current_assets,non_current_assets,current_liabilities,non_current_liabilities,equity,debt_details]

add_columns(dataframes,new_years)

In [6]:
#voy a tener que trasponer las matrices para operar más facilmente en pandas

income_statement=income_statement.transpose()
current_assets=current_assets.transpose()
non_current_assets=non_current_assets.transpose()
current_liabilities=current_liabilities.transpose()
non_current_liabilities=non_current_liabilities.transpose()
equity=equity.transpose()
debt_details=debt_details.transpose()

In [7]:
#Elimino las "," 

income_statement.replace(',','', regex=True, inplace=True)
current_assets.replace(',','', regex=True, inplace=True)
non_current_assets.replace(',','', regex=True, inplace=True)
current_liabilities.replace(',','', regex=True, inplace=True)
non_current_liabilities.replace(',','', regex=True, inplace=True)
equity.replace(',','', regex=True, inplace=True)
debt_details.replace(',','', regex=True, inplace=True)

In [8]:
cambio_a_float(income_statement)
cambio_a_float(current_assets)
cambio_a_float(non_current_assets)
cambio_a_float(current_liabilities)
cambio_a_float(non_current_liabilities)
cambio_a_float(equity)
cambio_a_float(debt_details)

In [9]:
#Esta incluye el 0 a añadir al principio del array cuando hagamos bop=eop(n-1)

zero=[0]

In [10]:
columns_to_keep=["Business Revenue","Cost of Goods and Services","Selling, General and Administrative Expenses","Depreciation and Amortization",
                "Interest Expense Net of Capitalized Interest","Interest Income","Non-Controlling/Minority Interests","profit","Total Assets",
                 "Cash, Cash Equivalents and Short Term Investments","Trade and Other Receivables, Current","Gross Property, Plant and Equipment",
                "Accumulated Depreciation and Impairment","Intangibles other than Goodwill","Accumulated Amortization and Impairment",
                "Deferred Tax Assets, Non-Current","Long Term Debt and Capital Lease Obligation","Tax Liabilities, Non-Current",
                "Trade and Other Payables, Current","Current Portion of Long Term Debt","Capital Lease Obligations, Current","Provisions, Current"
                "Deferred Liabilities, Current","Retained Earnings/Accumulated Deficit","Provision for Income Tax"]

In [11]:
#Para las columnas que no vamos a tocar vamos a rellenarlas con el valor del último año:
dataframes=[income_statement,current_assets,non_current_assets,current_liabilities,non_current_liabilities,equity,debt_details]

for i in income_statement.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            income_statement[i][j]=income_statement[i]["2021"]
        
for i in current_assets.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            current_assets[i][j]=current_assets[i]["2021"]
            
for i in non_current_assets.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            non_current_assets[i][j]=non_current_assets[i]["2021"]

for i in current_liabilities.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            current_liabilities[i][j]=current_liabilities[i]["2021"]
            
for i in non_current_liabilities.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            non_current_liabilities[i][j]=non_current_liabilities[i]["2021"]

for i in equity.columns:
    
    if i not in columns_to_keep:
        for j in new_years:
            
            equity[i][j]=equity[i]["2021"]
        

### Income statement (parts we can add)

In [12]:
forecasted_years=["2022E","2023E","2024E","2025E","2026E"]

In [13]:
#revenue: (usamos revenue growth)
start_val=income_statement["Business Revenue"]["2021"]

for i in forecasted_years:
    
    income_statement["Business Revenue"][i]=start_val*(1+sales_growth)
    
    start_val=start_val*(1+sales_growth)

In [14]:
#gross margin: (calculamos gross margin 2021 y mantenemos constante)

gross_profit=income_statement["Business Revenue"]["2021"]+income_statement["Cost of Goods and Services"]["2021"] #el cogs está en negativo
gross_margin=gross_profit/income_statement["Business Revenue"]["2021"]

gross_profit=[a*gross_margin for a in income_statement["Business Revenue"]]

In [15]:
#cogs: (calculamos a partir del gross margin y el revenue: Revenue-gross_profit)

income_statement["Cost of Goods and Services"]=[-(a-b) for a,b in zip(income_statement["Business Revenue"],gross_profit)]

In [16]:
# selling,general and administrative (calculamos sg&a as % of sales 2021 y mantenemos constante)

margin=-income_statement["Selling, General and Administrative Expenses"]["2021"]/income_statement["Business Revenue"]["2021"]
income_statement["Selling, General and Administrative Expenses"]=[-margin*a for a in income_statement["Business Revenue"]]

In [17]:
#Tax, expense (primero tengo que sacar el tax rate del último año para mantenerlo en el resto)

tax_rate=-income_statement["Provision for Income Tax"]["2021"]/(income_statement.sum(axis=1)["2021"]-income_statement["Provision for Income Tax"]["2021"])

#una vez tenemos el tax rate, el tax expense lo calcularemos más adelante

### Non-current assets calcs.

Para los non-current assets tenemos que hacer:

    Tanglibles:

    - PP&E-BoP-NET (+)CAPEX (-) Depreciation Expense
    
    Intangibles:
    
    -Intangibles-BoP-NET (+)CAPEX (-)Amortization Expense

In [18]:
#Tangibles: (añadimos a ppe en non_current_assets y a depreciation en non_current_assets)

ppe_bop=non_current_assets["Gross Property, Plant and Equipment"]["2021"]

for i in forecasted_years:

    non_current_assets["Gross Property, Plant and Equipment"][i]=ppe_bop+(capex_tangible*income_statement["Business Revenue"][i])-(depreciation_tangible*ppe_bop)
    non_current_assets["Accumulated Depreciation and Impairment"][i]=-depreciation_tangible*ppe_bop
    
    ppe_bop=non_current_assets["Gross Property, Plant and Equipment"][i]


In [19]:
#Inangibles: (añadimos a intangibles en non_current_assets y amortization en non_current_assets)

intangibles_bop=non_current_assets["Gross Property, Plant and Equipment"]["2021"]

for i in forecasted_years:

    non_current_assets["Intangibles other than Goodwill"][i]=intangibles_bop+(capex_tangible*income_statement["Business Revenue"][i])-(depreciation_tangible*intangibles_bop)
    non_current_assets["Accumulated Amortization and Impairment"][i]=-depreciation_tangible*intangibles_bop
    
    intangibles_bop=non_current_assets["Intangibles other than Goodwill"][i]


In [20]:
#sumamos amortization y depreciation y añadimos al income statement

income_statement["Depreciation and Amortization"]=non_current_assets["Accumulated Depreciation and Impairment"] + non_current_assets["Accumulated Amortization and Impairment"]

### Working capital calcs.

In [21]:
# Receivables

sales=list(income_statement["Business Revenue"])
dso=current_assets["Trade and Other Receivables, Current"]["2021"]/(income_statement["Business Revenue"]["2021"]/365) #dso= receivables/(sales/365)

#las dso(days sales outstanding) se mantienen a lo largo de los años por lo que con las sales y dso sacamos los receivables

receivables=[(a*dso)/365 for a in sales]  #añadir en current_assets en "Trade and Other Receivables, Current"

current_assets["Trade and Other Receivables, Current"]=receivables


In [22]:
#Inventories  (en este caso no hay inventories)

if "Inventories" in non_current_assets.columns:
    cogs=list(income_statement["Cost of Goods and Services"])
    dio=non_current_assets["inventories"][2021]/(income_statement["Cost of Goods and Services"]["2021"]/365) #dio= inventories/(COGS/365)

    inventories=[(a*dio)/365 for a in cogs]
    
    current_assets["Inventories"]=inventories

In [23]:
#payables 

#primero tenemos que sacar los purchases con los inventories
if "Inventories" in non_current_assets.columns:
    inventories_bop=list(non_current_assets["Inventories"].values)
    inventories_bop=zero+inventories_bop
    inventories_bop.pop()
    
    purchases=[a-b-c for a,b,c in zip(non_current_assets["inventories"],inventories_bop,income_statement["Cost of Goods and Services"])]
    
    dpo=current_liabilities["Trade and Other Payables, Current"]["2021"]/(purchases[-1] /365)
    payables=[(a*dpo)/365 for a in purchases]
    columns_to_add.append(payables)
    

else:
    purchases=0
    
    #en caso de que purchases=0 mantenemos el "Trade and Other Payables, Current" de 2021 para todos los años
    
    payables=[current_liabilities["Trade and Other Payables, Current"]["2021"] for a in current_liabilities["Trade and Other Payables, Current"]]
    
    current_liabilities["Trade and Other Payables, Current"]=payables

### Equity Calculations

In [24]:
income_statement["profit"]=income_statement.sum(axis=1) #para comprobar

**TENDREMOS QUE RELOAD AL FINAL YA QUE EL PROFIT IRÁ VARIANDO**

In [25]:
#parent equity 

parent_equity_bop=[a+b+c+d for a,b,c,d in zip(equity["Capital Stock"],equity["Treasury Stock"],equity["Retained Earnings/Accumulated Deficit"],equity["Reserves/Accumulated Comprehensive Income/Losses"])]
parent_equity_bop=zero+parent_equity_bop
parent_equity_bop.pop()

equity_attrib_to_parent=[i for i in income_statement.profit]

dividend=[-dividends*i for i in income_statement.profit]  #realmente tendriamos que cogerlo sobre el profit del año anterior 

parent_equity_eop=[a+b+c for a,b,c in zip(parent_equity_bop,equity_attrib_to_parent,dividend)]

In [26]:
equity["Retained Earnings/Accumulated Deficit"]=[a for a in parent_equity_eop]

Añadimos el parent equity a "Retained Earnings/Accumulated Deficit" y dejamos el resto de columnas como 0

### Debt Calculations

En el caso de la deuda, cada empresa lo reporta de una manera distinta. Para este caso vamos a hacerlo de dos maneras, una customizada a la empresa en cuestión ya que es la que me interesa, y otra general para que se pueda aplicar a cuclaquier empresa. El segundo método lo utilizamos ya que la idea es que nos de una idea general del precio de la empresa mas que un precio exacto.

#### Caso particular amcx

En este caso primero vamos a sumar los tipos de deuda por el "due date"

In [27]:
debt_types=["Debt due in Year 1","Debt due in Year 2","Debt due in Year 3","Debt due in Year 4","Debt due in Year 5"
            ,"Capital Lease due in Year 1","Capital Lease due in Year 2","Capital Lease due in Year 3"
            ,"Capital Lease due in Year 4","Capital Lease due in Year 5",]

In [28]:
#debt_year_2021, debt_year_2022, debt_year_2023, debt_year_2024, debt_year_2025=debt_due_year(debt_types,debt_details)

In [29]:
debt_maturities=debt_due_year(debt_types,debt_details)

In [30]:
total_debt_aot=non_current_liabilities["Long Term Debt and Capital Lease Obligation"]["2021"]+current_liabilities["Current Portion of Long Term Debt"]["2021"]+current_liabilities["Capital Lease Obligations, Current"]["2021"]

In [31]:
debt_bop=total_debt_aot
debt_eop=[]

for i in debt_maturities:

    debt_eop.append(debt_bop-i)
    debt_bop=debt_bop-i

In [32]:
#la short term debt de cada año será la maturity de la debt del año siguiente, calculamos y añadimos a current_liabilities

short_term_debt=list(debt_maturities[1:])
short_term_debt.append((debt_maturities)[-1])

a=0
for j in forecasted_years:
        
    current_liabilities["Current Portion of Long Term Debt"][j]=short_term_debt[a]
    a+=1

In [33]:
#long term debt será la total debt (dept_eop) menos la short term debt, calculamos y añadimos a non_current_liabilities

long_term_debt=[a-b for a,b in zip(debt_eop,short_term_debt)]

a=0
for j in forecasted_years:
        
    non_current_liabilities["Long Term Debt and Capital Lease Obligation"][j]=long_term_debt[a]
    a+=1

***El interest expense esperamos a añadirlo al final al income statement***

In [34]:
#calcular el interest rate del año anterior para sacar el interes expense

total_debt_2020=non_current_liabilities["Long Term Debt and Capital Lease Obligation"]["2020"]+current_liabilities["Current Portion of Long Term Debt"]["2021"]+current_liabilities["Capital Lease Obligations, Current"]["2020"]
promedio_debts=(total_debt_2020+total_debt_aot)/2   #promedio de la deuda total del ultimo año y del anterior
interest_rate=-income_statement["Interest Expense Net of Capitalized Interest"]["2021"]/promedio_debts 

#mantenemos el interest rate para todos los años. Por lo que el interest expense será el total debt por el interest rate

interest_expense=[a*interest_rate for a in debt_eop]


### Tax Calculations

In [45]:
tax_expense=income_statement["Provision for Income Tax"]
profit_before_tax=income_statement["profit"]-income_statement["Provision for Income Tax"]

In [46]:
income_statement

AMCX_income-statement_Annual_As_Originally_Reported,Business Revenue,Cost of Goods and Services,"Selling, General and Administrative Expenses",Depreciation and Amortization,Interest Expense Net of Capitalized Interest,Interest Income,Impairment/Write Off/Write Down of Other Assets,Gain/Loss on Extinguishment of Debt,Restructuring and Reorganization Income/Expense,Goodwill Impairment/Write Off,Litigation Income/Expense,"Other Income/Expense, Non-Operating",Provision for Income Tax,Non-Controlling/Minority Interests,profit
2018,2.971929e+09,-1.382908e+09,-8.611136e+08,-5.343790e+08,-154993000.0,19180000.0,-4486000.0,0.0,-45847000.0,0.0,0.0,29177000.0,-156306000.0,-17780000.0,-1.375267e+08
2019,3.060321e+09,-1.424039e+09,-8.867251e+08,-6.210230e+08,-157798000.0,24707000.0,0.0,0.0,-40914000.0,-106603000.0,0.0,-6000000.0,-78470000.0,-27230000.0,-2.637741e+08
2020,2.814956e+09,-1.309865e+09,-8.156308e+08,-6.578170e+08,-138610000.0,30032000.0,0.0,-2908000.0,-35068000.0,-122227000.0,0.0,71221000.0,-145391000.0,-17009000.0,-3.283167e+08
2021,3.077608e+09,-1.432083e+09,-8.917340e+08,-7.135750e+08,-129073000.0,10243000.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,-94393000.0,-29243000.0,-3.690980e+08
2022E,3.323817e+09,-1.546650e+09,-9.630727e+08,-7.653480e+07,0.0,0.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,0.0,0.0,5.707115e+08
2023E,3.589722e+09,-1.670382e+09,-1.040119e+09,-9.188561e+07,0.0,0.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,0.0,0.0,6.204882e+08
2024E,3.876900e+09,-1.804012e+09,-1.123328e+09,-1.079108e+08,0.0,0.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,0.0,0.0,6.748008e+08
2025E,4.187052e+09,-1.948333e+09,-1.213194e+09,-1.246976e+08,0.0,0.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,0.0,0.0,7.339788e+08
2026E,4.522016e+09,-2.104200e+09,-1.310250e+09,-1.423380e+08,0.0,0.0,-159610000.0,-22074000.0,-10378000.0,0.0,0.0,25214000.0,0.0,0.0,7.983803e+08


#### Tax assets

In [48]:
deferred_tax_assets_bop=non_current_assets["Deferred Tax Assets, Non-Current"]["2021"]

tax_assets_list=[]

for i in forecasted_years:
    while deferred_tax_assets_bop >0:
        non_current_assets["Deferred Tax Assets, Non-Current"][i]=deferred_tax_assets_bop-income_statement.sum(axis=1)[i]-income_statement["Provision for Income Tax"][i]
        deferred_tax_assets_bop=tax_assets
        
        if tax_assets<0:
            
            tax_assets=0
            

TypeError: '>' not supported between instances of 'list' and 'int'

### Cash Flow 

#### Cash Flow from Operations


Net Income

(+) D&A expense  (income statement)

(+/-) Change in DTAs         **los sacamos una vez tengamos los taxes**

(-) Net working capital

    CA.- ex. cash
    
    CL.- ex. Interest bearing debt
    
    WC Balance
   
    
=Total cash flow from operations
    
    

In [39]:
current_assets_wc_balance=current_assets.drop(["Total Assets","Cash, Cash Equivalents and Short Term Investments"], axis=1).sum(axis=1)
current_liabilities_wc_balance=current_liabilities.drop("Current Portion of Long Term Debt",axis=1).sum(axis=1)

WC_balance=[a-b for a,b in zip(current_assets_wc_balance,current_liabilities_wc_balance)]